In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from video_moment_retrieval.utils.logging import init_logging, logger
from video_moment_retrieval.datasets.qv_highlights import QVDataset, pad_collate
from video_moment_retrieval.testing.double_decoder import DoubleDecoder, DoubleDecoderConfig, DDDataset
from video_moment_retrieval.moment_detr.model import VideoDetrConfig, MomentDetr
from transformers import TrainingArguments, Trainer, EvalPrediction, PreTrainedTokenizerFast
from tokenizers import Tokenizer

init_logging()

/home/ioan/.cache/pypoetry/virtualenvs/video-moment-retrieval-rdzoMJ72-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=Tokenizer.from_file("video_moment_retrieval/testing/tokenizer.json"),
    pad_token="[PAD]",
    unk_token="[UNK]"
)
train_dataset = DDDataset("qvhighlights_features/highlight_train_release.jsonl", "qvhighlights_features/bert_features", "qvhighlights_features/resnet_features", target_tokenizer=tokenizer)
eval_dataset = DDDataset("qvhighlights_features/highlight_val_release.jsonl", "qvhighlights_features/bert_features", "qvhighlights_features/resnet_features", target_tokenizer=tokenizer)

config = DoubleDecoderConfig(
    len(tokenizer.vocab),
    tokenizer.pad_token_id,
)
# logger.info("Running model using config %s", config)

model = DoubleDecoder(config)

train_args = TrainingArguments(
    "./train_output/double_decoder",
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=1e-4,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=500,
    num_train_epochs=200,
    save_steps=1000,
    eval_strategy="steps",
    eval_steps=1000,
    load_best_model_at_end=True,
    greater_is_better=False,
    max_grad_norm=0.1,
    label_names=["labels"],
    weight_decay=1e-4,
    eval_do_concat_batches=False,
    dataloader_num_workers=4,
    # metric_for_best_model="mAP"
    # use_cpu=True
)

trainer = Trainer(
    model=model,
    args=train_args,
    data_collator=train_dataset.collate,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    # compute_metrics=compute_metrics
)

trainer.train()

/home/ioan/.cache/pypoetry/virtualenvs/video-moment-retrieval-rdzoMJ72-py3.10/lib/python3.10/site-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss
1000,1.442300,1.529170
2000,0.534900,1.916914
3000,0.237000,2.196502
4000,0.131400,2.628696


/home/ioan/.cache/pypoetry/virtualenvs/video-moment-retrieval-rdzoMJ72-py3.10/lib/python3.10/site-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/home/ioan/.cache/pypoetry/virtualenvs/video-moment-retrieval-rdzoMJ72-py3.10/lib/python3.10/site-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/home/ioan/.cache/pypoetry/virtualenvs/video-moment-retrieval-rdzoMJ72-py3.10/lib/python3.10/site-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/home/ioan/.cache/pypoetry/virtualenvs/video-moment-retrieval-rdzoMJ72-py3.10/lib/python3.10/site-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_m

: 